In [ ]:
pip install requests pandas numpy matplotlib seaborn scipy requests-cache retry-requests

In [4]:
# ======== IMPORTS ========
import os
import sys
import time
import math
import random
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from scipy import stats

In [6]:
# VS Code script to analyze missing values for 2015, 2016, and 2017 Visual Crossing weather datasets

import pandas as pd
import os

# === Config ===
YEARS = [2015, 2016, 2017]
CHUNKSIZE = 200000  # Safe size for large CSVs
INPUT_DIR = "."

def check_missing_values(file_path):
    total_counts = None
    missing_counts = None

    for chunk in pd.read_csv(file_path, chunksize=CHUNKSIZE, low_memory=False):
        if total_counts is None:
            total_counts = chunk.count()
            missing_counts = chunk.isnull().sum()
        else:
            total_counts += chunk.count()
            missing_counts += chunk.isnull().sum()

    # Estimate total rows (average across columns)
    total_rows = sum(total_counts) / len(total_counts)
    
    # Build report
    missing_report = pd.DataFrame({
        "Missing Values": missing_counts,
        "Missing %": (missing_counts / total_rows * 100).round(2)
    }).sort_values(by="Missing %", ascending=False)

    return missing_report

if __name__ == "__main__":
    for year in YEARS:
        file_name = f"visualcrossing_weather_{year}.csv"
        if not os.path.exists(file_name):
            print(f"Skipping {year} - file not found: {file_name}")
            continue
        
        print(f"\n=== Processing {file_name} ===")
        report = check_missing_values(file_name)
        print(report)
        
        # Save a report for each year
        output_name = f"Weather_Missing_Report_{year}.csv"
        report.to_csv(output_name)
        print(f"Missing value report saved as {output_name}")




=== Processing visualcrossing_weather_2015.csv ===
                Missing Values  Missing %
preciptype              657199      50.75
solarradiation           48910       3.78
tempmin                      0       0.00
temp                         0       0.00
datetime                     0       0.00
tempmax                      0       0.00
precip                       0       0.00
humidity                     0       0.00
windspeed                    0       0.00
snow                         0       0.00
cloudcover                   0       0.00
latitude                     0       0.00
longitude                    0       0.00
year                         0       0.00
Missing value report saved as Weather_Missing_Report_2015.csv

=== Processing visualcrossing_weather_2016.csv ===
                Missing Values  Missing %
preciptype              495070      49.06
solarradiation           58194       5.77
tempmin                      0       0.00
temp                         0      

In [7]:
# Cleans Visual Crossing weather data (2015–2017):
# - Fills missing solarradiation with hybrid method (interpolation → rolling mean → ffill/bfill)
# - Fills preciptype using temperature + precipitation rules
# - Adds binary flags for rain, snow, freezingrain
# - Outputs cleaned weather files for each year

import pandas as pd
import os

# === Configuration ===
YEARS = [2015, 2016, 2017]
INPUT_DIR = "."   
OUTPUT_DIR = "./cleaned_weather"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Column names
DATE_COL = "datetime"
LAT_COL = "latitude"
LON_COL = "longitude"
SOLAR_COL = "solarradiation"
PRECIP_COL = "precip"
TEMP_COL = "temp"
PRECIPTYPE_COL = "preciptype"

# === Step 1: Hybrid Fill for Solar Radiation ===
def fill_solar(group):
    # Linear interpolation first (handles most gaps)
    group[SOLAR_COL] = group[SOLAR_COL].interpolate(method='linear', limit_direction='both')
    
    # Rolling mean (7-day window) for any remaining NaNs
    group[SOLAR_COL] = group[SOLAR_COL].fillna(group[SOLAR_COL].rolling(window=7, min_periods=1).mean())
    
    # Forward/backward fill as final fallback (edges only)
    group[SOLAR_COL] = group[SOLAR_COL].fillna(method='ffill').fillna(method='bfill')
    return group

# === Step 2: Fill Precipitation Type (Categorical) ===
def infer_preciptype(row):
    # Keep existing label if present
    if pd.notnull(row[PRECIPTYPE_COL]):
        return row[PRECIPTYPE_COL]
    
    # If no precipitation
    if pd.isnull(row[PRECIP_COL]) or row[PRECIP_COL] == 0:
        return "none"
    
    # Infer based on temperature
    if row[TEMP_COL] < 0:
        return "snow"
    elif 0 <= row[TEMP_COL] <= 2:
        return "freezingrain"
    else:
        return "rain"

# === Main Cleaning Function ===
def clean_weather_file(year):
    input_file = os.path.join(INPUT_DIR, f"visualcrossing_weather_{year}.csv")
    output_file = os.path.join(OUTPUT_DIR, f"visualcrossing_weather_{year}_cleaned.csv")
    
    print(f"Processing {year}...")
    df = pd.read_csv(input_file, parse_dates=[DATE_COL])
    df = df.sort_values(by=[LAT_COL, LON_COL, DATE_COL])
    
    # Fill solar radiation with hybrid method
    df = df.groupby([LAT_COL, LON_COL], group_keys=False).apply(fill_solar)
    
    # Fill preciptype using rules
    df[PRECIPTYPE_COL] = df.apply(infer_preciptype, axis=1)
    df[PRECIPTYPE_COL] = df[PRECIPTYPE_COL].fillna("unknown")
    
    # Add binary weather flags for ML/RL features
    df["is_rain"] = (df[PRECIPTYPE_COL] == "rain").astype(int)
    df["is_snow"] = (df[PRECIPTYPE_COL] == "snow").astype(int)
    df["is_freezingrain"] = (df[PRECIPTYPE_COL] == "freezingrain").astype(int)
    
    # Save cleaned file
    df.to_csv(output_file, index=False)
    print(f"Saved cleaned weather file for {year}: {output_file}")

if __name__ == "__main__":
    for year in YEARS:
        clean_weather_file(year)
    print("\nAll weather files (2015–2017) have been cleaned and saved in ./cleaned_weather")

Processing 2015...


C:\Users\Zaka and Family\AppData\Local\Temp\ipykernel_21848\3203080287.py:34: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  group[SOLAR_COL] = group[SOLAR_COL].fillna(method='ffill').fillna(method='bfill')
C:\Users\Zaka and Family\AppData\Local\Temp\ipykernel_21848\3203080287.py:65: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby([LAT_COL, LON_COL], group_keys=False).apply(fill_solar)


Saved cleaned weather file for 2015: ./cleaned_weather\visualcrossing_weather_2015_cleaned.csv
Processing 2016...


C:\Users\Zaka and Family\AppData\Local\Temp\ipykernel_21848\3203080287.py:34: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  group[SOLAR_COL] = group[SOLAR_COL].fillna(method='ffill').fillna(method='bfill')
C:\Users\Zaka and Family\AppData\Local\Temp\ipykernel_21848\3203080287.py:65: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby([LAT_COL, LON_COL], group_keys=False).apply(fill_solar)


Saved cleaned weather file for 2016: ./cleaned_weather\visualcrossing_weather_2016_cleaned.csv
Processing 2017...


C:\Users\Zaka and Family\AppData\Local\Temp\ipykernel_21848\3203080287.py:34: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  group[SOLAR_COL] = group[SOLAR_COL].fillna(method='ffill').fillna(method='bfill')
C:\Users\Zaka and Family\AppData\Local\Temp\ipykernel_21848\3203080287.py:65: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby([LAT_COL, LON_COL], group_keys=False).apply(fill_solar)


Saved cleaned weather file for 2017: ./cleaned_weather\visualcrossing_weather_2017_cleaned.csv

All weather files (2015–2017) have been cleaned and saved in ./cleaned_weather


In [8]:
# group Data_Co.csv by day for merging with daily weather data 

import pandas as pd

# === Configuration ===
INPUT_FILE = "Data_Co_cleaned.csv"          # Path to raw DataCo dataset
OUTPUT_FILE = "Data_Co_Daily.csv"   # Output aggregated file
DATE_COLUMN = "order_date"          

# === Step 1: Load the data (handles large files in chunks if needed) ===
def load_data(file_path, chunksize=100000):
    chunks = []
    for chunk in pd.read_csv(file_path, chunksize=chunksize, low_memory=False):
        chunks.append(chunk)
    return pd.concat(chunks, ignore_index=True)

# === Step 2: Process and group by day ===
def group_by_day(df, date_col):
    # Ensure date column is in datetime format
    df[date_col] = pd.to_datetime(df[date_col], errors='coerce')

    # Drop rows where date couldn't be parsed
    df = df.dropna(subset=[date_col])

    # Extract just the date (no time)
    df["date_only"] = df[date_col].dt.date

    # Group by day, summing numerical columns (e.g., sales, quantity)
    daily_df = df.groupby("date_only").sum(numeric_only=True).reset_index()

    return daily_df

# === Step 3: Save the result ===
def save_data(df, output_file):
    df.to_csv(output_file, index=False)
    print(f"Saved aggregated daily data to: {output_file}")

# === Main ===
if __name__ == "__main__":
    print("Loading data...")
    data = load_data(INPUT_FILE)

    print("Aggregating by day...")
    daily_data = group_by_day(data, DATE_COLUMN)

    print("Saving aggregated data...")
    save_data(daily_data, OUTPUT_FILE)


Loading data...
Aggregating by day...
Saving aggregated data...
Saved aggregated daily data to: Data_Co_Daily.csv


In [9]:
# Merges Data_Co_Daily.csv with Visual Crossing weather files (2015–2017)
# Standardizes date & coordinate columns before merging

import pandas as pd
import os

# === Config ===
SALES_FILE = "Data_Co_Daily.csv"  # Daily aggregated sales (all years)
WEATHER_DIR = "./cleaned_weather"  # Where cleaned weather files are stored
OUTPUT_DIR = "./merged_data"
os.makedirs(OUTPUT_DIR, exist_ok=True)

YEARS = [2015, 2016, 2017]

# Column names
SALES_DATE_COL = "date_only"
WEATHER_DATE_COL = "datetime"
LAT_SALES = "Latitude"
LON_SALES = "Longitude"
LAT_WEATHER = "latitude"
LON_WEATHER = "longitude"

# Load sales data
print("Loading sales data...")
sales_df = pd.read_csv(SALES_FILE)

# Convert date to datetime object (normalize formats)
sales_df[SALES_DATE_COL] = pd.to_datetime(sales_df[SALES_DATE_COL], format="%m/%d/%Y", errors='coerce')

# Standardize coordinate names in sales
sales_df = sales_df.rename(columns={LAT_SALES: "latitude", LON_SALES: "longitude"})

# Process year by year
for year in YEARS:
    weather_file = os.path.join(WEATHER_DIR, f"visualcrossing_weather_{year}_cleaned.csv")
    if not os.path.exists(weather_file):
        print(f"Weather file missing for {year}: {weather_file}")
        continue

    print(f"\nMerging {year} sales with weather...")
    weather_df = pd.read_csv(weather_file)
    
    # Convert weather date
    weather_df[WEATHER_DATE_COL] = pd.to_datetime(weather_df[WEATHER_DATE_COL], format="%m/%d/%Y", errors='coerce')

    # Filter sales for this year only
    sales_year = sales_df[sales_df[SALES_DATE_COL].dt.year == year].copy()

    # Merge on date + coordinates
    merged = pd.merge(
        sales_year,
        weather_df,
        left_on=[SALES_DATE_COL, "latitude", "longitude"],
        right_on=[WEATHER_DATE_COL, "latitude", "longitude"],
        how="left"
    )

    # Drop duplicate date column
    merged = merged.drop(columns=[WEATHER_DATE_COL])

    # Save merged file
    output_file = os.path.join(OUTPUT_DIR, f"DataCo_Weather_{year}_Merged.csv")
    merged.to_csv(output_file, index=False)
    print(f"Saved merged dataset for {year}: {output_file}")


Loading sales data...

Merging 2015 sales with weather...
Saved merged dataset for 2015: ./merged_data\DataCo_Weather_2015_Merged.csv

Merging 2016 sales with weather...
Saved merged dataset for 2016: ./merged_data\DataCo_Weather_2016_Merged.csv

Merging 2017 sales with weather...
Saved merged dataset for 2017: ./merged_data\DataCo_Weather_2017_Merged.csv


In [10]:
# Debug script to identify why merged files are empty

import pandas as pd
import os

# === Config ===
SALES_FILE = "Data_Co_Daily.csv"
WEATHER_DIR = "./cleaned_weather"
YEARS = [2015, 2016, 2017]

# Column names
SALES_DATE_COL = "date_only"
WEATHER_DATE_COL = "datetime"
LAT_SALES = "Latitude"
LON_SALES = "Longitude"

# Load sales data
print("Loading sales data...")
sales_df = pd.read_csv(SALES_FILE)

# Convert date to datetime
sales_df[SALES_DATE_COL] = pd.to_datetime(sales_df[SALES_DATE_COL], format="%m/%d/%Y", errors='coerce')

# Rename lat/lon to match weather files
sales_df = sales_df.rename(columns={LAT_SALES: "latitude", LON_SALES: "longitude"})

# Check overall sales summary
print("\n=== Sales Data Info ===")
print(f"Rows: {sales_df.shape[0]}")
print(f"Unique dates: {sales_df[SALES_DATE_COL].nunique()}")
print(f"Unique lat/lon pairs: {sales_df[['latitude','longitude']].drop_duplicates().shape[0]}")
print("Sample sales dates:", sales_df[SALES_DATE_COL].dropna().unique()[:5])
print("Sample sales coords:\n", sales_df[['latitude','longitude']].drop_duplicates().head())

for year in YEARS:
    weather_file = os.path.join(WEATHER_DIR, f"visualcrossing_weather_{year}_cleaned.csv")
    if not os.path.exists(weather_file):
        print(f"\nSkipping {year} - weather file not found")
        continue

    print(f"\n=== Debugging Year {year} ===")
    weather_df = pd.read_csv(weather_file)

    # Convert weather date
    weather_df[WEATHER_DATE_COL] = pd.to_datetime(weather_df[WEATHER_DATE_COL], format="%m/%d/%Y", errors='coerce')

    # Show weather summary
    print(f"Weather Rows: {weather_df.shape[0]}")
    print(f"Unique dates: {weather_df[WEATHER_DATE_COL].nunique()}")
    print(f"Unique lat/lon pairs: {weather_df[['latitude','longitude']].drop_duplicates().shape[0]}")
    print("Sample weather dates:", weather_df[WEATHER_DATE_COL].dropna().unique()[:5])
    print("Sample weather coords:\n", weather_df[['latitude','longitude']].drop_duplicates().head())

    # Check intersection of dates
    common_dates = set(sales_df[SALES_DATE_COL].dt.date).intersection(set(weather_df[WEATHER_DATE_COL].dt.date))
    print(f"Common dates between sales & weather ({year}): {len(common_dates)}")

    # Check intersection of coordinates
    sales_coords = set([tuple(x) for x in sales_df[['latitude','longitude']].drop_duplicates().values])
    weather_coords = set([tuple(x) for x in weather_df[['latitude','longitude']].drop_duplicates().values])
    common_coords = sales_coords.intersection(weather_coords)
    print(f"Common lat/lon pairs between sales & weather ({year}): {len(common_coords)}")

    # If needed, list some sample common values
    if common_dates:
        print("Sample common date:", list(common_dates)[:3])
    if common_coords:
        print("Sample common coords:", list(common_coords)[:3])

    if not common_dates or not common_coords:
        print(f"⚠️ No matches for {year}! Likely date format or coordinate mismatch.")


Loading sales data...

=== Sales Data Info ===
Rows: 1127
Unique dates: 0
Unique lat/lon pairs: 1127
Sample sales dates: <DatetimeArray>
[]
Length: 0, dtype: datetime64[ns]
Sample sales coords:
       latitude     longitude
0  4896.563650 -13865.259369
1  5034.414862 -13375.784661
2  5440.104700 -14952.370637
3  5239.624569 -15943.433639
4  4344.302576 -13448.148567

=== Debugging Year 2015 ===
Weather Rows: 1345390
Unique dates: 0
Unique lat/lon pairs: 3686
Sample weather dates: <DatetimeArray>
[]
Length: 0, dtype: datetime64[ns]
Sample weather coords:
        latitude  longitude
0     17.982491 -66.112671
365   18.006821 -66.635864
730   18.018362 -66.616234
1095  18.025232 -66.613274
1460  18.025280 -66.613068
Common dates between sales & weather (2015): 1
Common lat/lon pairs between sales & weather (2015): 0
Sample common date: [NaT]
⚠️ No matches for 2015! Likely date format or coordinate mismatch.

=== Debugging Year 2016 ===
Weather Rows: 1048575
Unique dates: 0
Unique lat/lon 

In [11]:
# Debug + Fix script to merge Data_Co_Daily.csv with Visual Crossing weather data (2015–2017)
# - Prints debug info (date & coordinate mismatches)
# - Normalizes dates and rounds coordinates
# - Outputs merged files for each year

import pandas as pd
import os

# === Config ===
SALES_FILE = "Data_Co_Daily.csv"
WEATHER_DIR = "./cleaned_weather"   # Cleaned weather files for 2015–2017
OUTPUT_DIR = "./merged_data"
os.makedirs(OUTPUT_DIR, exist_ok=True)

YEARS = [2015, 2016, 2017]

# Column names
SALES_DATE_COL = "date_only"
WEATHER_DATE_COL = "datetime"
LAT_SALES = "Latitude"
LON_SALES = "Longitude"

# === Load and preprocess sales data ===
print("Loading sales data...")
sales_df = pd.read_csv(SALES_FILE)

# Convert date to datetime, normalize to YYYY-MM-DD
sales_df[SALES_DATE_COL] = pd.to_datetime(sales_df[SALES_DATE_COL], format="%m/%d/%Y", errors='coerce').dt.normalize()

# Rename coordinates to match weather
sales_df = sales_df.rename(columns={LAT_SALES: "latitude", LON_SALES: "longitude"})

# Debug sales summary
print("\n=== Sales Data Summary ===")
print(f"Rows: {sales_df.shape[0]}")
print(f"Unique dates: {sales_df[SALES_DATE_COL].nunique()}")
print(f"Unique lat/lon pairs: {sales_df[['latitude','longitude']].drop_duplicates().shape[0]}")
print("Sample sales dates:", sales_df[SALES_DATE_COL].dropna().dt.strftime('%Y-%m-%d').unique()[:5])
print("Sample sales coords:\n", sales_df[['latitude','longitude']].drop_duplicates().head())

for year in YEARS:
    weather_file = os.path.join(WEATHER_DIR, f"visualcrossing_weather_{year}_cleaned.csv")
    if not os.path.exists(weather_file):
        print(f"\nSkipping {year} - weather file not found")
        continue

    print(f"\n=== Processing Year {year} ===")
    weather_df = pd.read_csv(weather_file)

    # Convert weather date, normalize
    weather_df[WEATHER_DATE_COL] = pd.to_datetime(weather_df[WEATHER_DATE_COL], format="%m/%d/%Y", errors='coerce').dt.normalize()

    # Debug weather summary
    print(f"Weather Rows: {weather_df.shape[0]}")
    print(f"Unique dates: {weather_df[WEATHER_DATE_COL].nunique()}")
    print(f"Unique lat/lon pairs: {weather_df[['latitude','longitude']].drop_duplicates().shape[0]}")
    print("Sample weather dates:", weather_df[WEATHER_DATE_COL].dropna().dt.strftime('%Y-%m-%d').unique()[:5])
    print("Sample weather coords:\n", weather_df[['latitude','longitude']].drop_duplicates().head())

    # Step 1: Check raw intersections
    raw_dates = set(sales_df[SALES_DATE_COL].dt.date)
    raw_weather_dates = set(weather_df[WEATHER_DATE_COL].dt.date)
    common_dates_raw = raw_dates.intersection(raw_weather_dates)

    sales_coords_raw = set([tuple(x) for x in sales_df[['latitude','longitude']].drop_duplicates().values])
    weather_coords_raw = set([tuple(x) for x in weather_df[['latitude','longitude']].drop_duplicates().values])
    common_coords_raw = sales_coords_raw.intersection(weather_coords_raw)

    print(f"Raw common dates: {len(common_dates_raw)}")
    print(f"Raw common coords: {len(common_coords_raw)}")

    # Step 2: Fix coordinates by rounding to 3 decimals
    sales_df["latitude"] = sales_df["latitude"].round(3)
    sales_df["longitude"] = sales_df["longitude"].round(3)
    weather_df["latitude"] = weather_df["latitude"].round(3)
    weather_df["longitude"] = weather_df["longitude"].round(3)

    sales_coords = set([tuple(x) for x in sales_df[['latitude','longitude']].drop_duplicates().values])
    weather_coords = set([tuple(x) for x in weather_df[['latitude','longitude']].drop_duplicates().values])
    common_coords = sales_coords.intersection(weather_coords)

    print(f"Common coords (after rounding): {len(common_coords)}")
    if common_coords:
        print("Sample common coords:", list(common_coords)[:5])

    # Step 3: Merge (year-specific sales only)
    sales_year = sales_df[sales_df[SALES_DATE_COL].dt.year == year].copy()

    merged = pd.merge(
        sales_year,
        weather_df,
        left_on=[SALES_DATE_COL, "latitude", "longitude"],
        right_on=[WEATHER_DATE_COL, "latitude", "longitude"],
        how="left"
    )

    # Drop duplicate date column from weather
    merged = merged.drop(columns=[WEATHER_DATE_COL])

    # Save merged dataset
    output_file = os.path.join(OUTPUT_DIR, f"DataCo_Weather_{year}_Merged.csv")
    merged.to_csv(output_file, index=False)
    print(f"Saved merged file for {year}: {output_file}")
    print(f"Final merged rows for {year}: {merged.shape[0]}")


Loading sales data...

=== Sales Data Summary ===
Rows: 1127
Unique dates: 0
Unique lat/lon pairs: 1127
Sample sales dates: []
Sample sales coords:
       latitude     longitude
0  4896.563650 -13865.259369
1  5034.414862 -13375.784661
2  5440.104700 -14952.370637
3  5239.624569 -15943.433639
4  4344.302576 -13448.148567

=== Processing Year 2015 ===
Weather Rows: 1345390
Unique dates: 0
Unique lat/lon pairs: 3686
Sample weather dates: []
Sample weather coords:
        latitude  longitude
0     17.982491 -66.112671
365   18.006821 -66.635864
730   18.018362 -66.616234
1095  18.025232 -66.613274
1460  18.025280 -66.613068
Raw common dates: 1
Raw common coords: 0
Common coords (after rounding): 0
Saved merged file for 2015: ./merged_data\DataCo_Weather_2015_Merged.csv
Final merged rows for 2015: 0

=== Processing Year 2016 ===
Weather Rows: 1048575
Unique dates: 0
Unique lat/lon pairs: 2865
Sample weather dates: []
Sample weather coords:
        latitude  longitude
0     17.982491 -66.11

In [12]:
# Inspects and fixes Data_Co_Daily.csv (date format & coordinates) before merging with weather files (2015–2017)

import pandas as pd
import numpy as np
import os
import math

# === Config ===
SALES_FILE = "Data_Co_Daily.csv"
WEATHER_DIR = "./cleaned_weather"
OUTPUT_DIR = "./merged_data_fixed"
os.makedirs(OUTPUT_DIR, exist_ok=True)

YEARS = [2015, 2016, 2017]
SALES_DATE_COL = "date_only"
WEATHER_DATE_COL = "datetime"
LAT_COL = "Latitude"
LON_COL = "Longitude"

# --- Helper: Convert projected (EPSG:3857 Web Mercator) coords to lat/lon ---
def mercator_to_latlon(x, y):
    lon = x / 20037508.34 * 180
    lat = y / 20037508.34 * 180
    lat = 180 / math.pi * (2 * math.atan(math.exp(lat * math.pi / 180)) - math.pi / 2)
    return lat, lon

# --- Load Sales Data ---
print("Loading sales data...")
sales_df = pd.read_csv(SALES_FILE)

# Inspect raw date and coordinates
print("\n=== Raw Data Samples (before parsing) ===")
print("Sample date_only values:", sales_df[SALES_DATE_COL].head().tolist())
print("Sample Latitude/Longitude values:", sales_df[[LAT_COL, LON_COL]].head().values.tolist())

# Try parsing date
sales_df[SALES_DATE_COL] = pd.to_datetime(sales_df[SALES_DATE_COL], errors='coerce')

# If all failed, try alternative format (day-first)
if sales_df[SALES_DATE_COL].isna().all():
    sales_df[SALES_DATE_COL] = pd.to_datetime(sales_df[SALES_DATE_COL], format="%d/%m/%Y", errors='coerce')

# Normalize dates (remove time)
sales_df[SALES_DATE_COL] = sales_df[SALES_DATE_COL].dt.normalize()

# Check if coordinates look like degrees (valid ranges: lat -90 to 90, lon -180 to 180)
if (sales_df[LAT_COL].abs() > 90).any() or (sales_df[LON_COL].abs() > 180).any():
    print("\nDetected projected coordinates (not degrees) — converting from Web Mercator (EPSG:3857) to lat/lon...")
    latlon = sales_df.apply(lambda r: mercator_to_latlon(r[LON_COL], r[LAT_COL]), axis=1)
    sales_df["latitude"] = [p[0] for p in latlon]
    sales_df["longitude"] = [p[1] for p in latlon]
else:
    sales_df = sales_df.rename(columns={LAT_COL: "latitude", LON_COL: "longitude"})

# Check fixed samples
print("\n=== Fixed Data Samples (after conversion) ===")
print("Parsed dates sample:", sales_df[SALES_DATE_COL].dropna().unique()[:5])
print("Latitude/Longitude sample:", sales_df[['latitude','longitude']].head())

# --- Merge Year by Year ---
for year in YEARS:
    weather_file = os.path.join(WEATHER_DIR, f"visualcrossing_weather_{year}_cleaned.csv")
    if not os.path.exists(weather_file):
        print(f"\nSkipping {year} - weather file not found")
        continue

    weather_df = pd.read_csv(weather_file)
    weather_df[WEATHER_DATE_COL] = pd.to_datetime(weather_df[WEATHER_DATE_COL], errors='coerce').dt.normalize()

    # Round coordinates for matching
    sales_df["latitude"] = sales_df["latitude"].round(3)
    sales_df["longitude"] = sales_df["longitude"].round(3)
    weather_df["latitude"] = weather_df["latitude"].round(3)
    weather_df["longitude"] = weather_df["longitude"].round(3)

    # Filter sales for current year
    sales_year = sales_df[sales_df[SALES_DATE_COL].dt.year == year].copy()

    # Debug intersections
    common_dates = set(sales_year[SALES_DATE_COL].dt.date).intersection(set(weather_df[WEATHER_DATE_COL].dt.date))
    sales_coords = set([tuple(x) for x in sales_year[['latitude','longitude']].drop_duplicates().values])
    weather_coords = set([tuple(x) for x in weather_df[['latitude','longitude']].drop_duplicates().values])
    common_coords = sales_coords.intersection(weather_coords)

    print(f"\n=== Year {year} Debug ===")
    print(f"Sales rows for {year}: {sales_year.shape[0]}")
    print(f"Weather rows: {weather_df.shape[0]}")
    print(f"Common dates: {len(common_dates)}")
    print(f"Common coords (after rounding): {len(common_coords)}")

    # Merge
    merged = pd.merge(
        sales_year,
        weather_df,
        left_on=[SALES_DATE_COL, "latitude", "longitude"],
        right_on=[WEATHER_DATE_COL, "latitude", "longitude"],
        how="left"
    )
    merged = merged.drop(columns=[WEATHER_DATE_COL])

    # Save output
    output_file = os.path.join(OUTPUT_DIR, f"DataCo_Weather_{year}_Merged.csv")
    merged.to_csv(output_file, index=False)
    print(f"Saved merged dataset for {year}: {output_file} (Rows: {merged.shape[0]})")


Loading sales data...

=== Raw Data Samples (before parsing) ===
Sample date_only values: ['2015-01-01', '2015-01-02', '2015-01-03', '2015-01-04', '2015-01-05']
Sample Latitude/Longitude values: [[4896.56365017, -13865.25936908], [5034.41486184, -13375.78466069], [5440.1047001, -14952.37063659], [5239.6245690000005, -15943.43363927], [4344.30257636, -13448.14856739]]

Detected projected coordinates (not degrees) — converting from Web Mercator (EPSG:3857) to lat/lon...

=== Fixed Data Samples (after conversion) ===
Parsed dates sample: <DatetimeArray>
['2015-01-01 00:00:00', '2015-01-02 00:00:00', '2015-01-03 00:00:00',
 '2015-01-04 00:00:00', '2015-01-05 00:00:00']
Length: 5, dtype: datetime64[ns]
Latitude/Longitude sample:    latitude  longitude
0  0.043987  -0.124554
1  0.045225  -0.120157
2  0.048869  -0.134319
3  0.047068  -0.143222
4  0.039026  -0.120807

=== Year 2015 Debug ===
Sales rows for 2015: 365
Weather rows: 1345390
Common dates: 365
Common coords (after rounding): 0
Save

In [1]:
# Aggregates Data_Co.csv by day, preserving proper lat/lon in degrees.

import pandas as pd

INPUT_FILE = "Data_Co_cleaned.csv"
OUTPUT_FILE = "Data_Co_Daily_new.csv"
DATE_COLUMN = "order_date"

print("Loading raw DataCo data...")
df = pd.read_csv(INPUT_FILE)

# Convert order date
df[DATE_COLUMN] = pd.to_datetime(df[DATE_COLUMN], errors='coerce')

# Drop rows with invalid dates
df = df.dropna(subset=[DATE_COLUMN])

# Extract date only
df["date_only"] = df[DATE_COLUMN].dt.date

# Group by date, latitude, longitude (keeping coordinates in degrees)
agg_cols = {
    col: 'sum' for col in df.columns if col not in [DATE_COLUMN, "date_only", "Latitude", "Longitude"]
}
# Keep first lat/lon per group (assuming consistent location per row)
agg_cols["Latitude"] = 'first'
agg_cols["Longitude"] = 'first'

df_daily = df.groupby(["date_only"], as_index=False).agg(agg_cols)

df_daily.to_csv(OUTPUT_FILE, index=False)
print(f"Saved fixed daily aggregated file with proper lat/lon: {OUTPUT_FILE}")


Loading raw DataCo data...
Saved fixed daily aggregated file with proper lat/lon: Data_Co_Daily_new.csv


In [2]:
# Merges cleaned Data_Co_Daily_new.csv (lat/lon in degrees) with Visual Crossing weather files (2015–2017)

import pandas as pd
import os

# === Config ===
SALES_FILE = "Data_Co_Daily_new.csv"      # Daily aggregated sales with degrees lat/lon
WEATHER_DIR = "./cleaned_weather"     # Cleaned weather files for 2015–2017
OUTPUT_DIR = "./merged_data_final"    # Output folder
os.makedirs(OUTPUT_DIR, exist_ok=True)

YEARS = [2015, 2016, 2017]

# Column names
SALES_DATE_COL = "date_only"
WEATHER_DATE_COL = "datetime"
LAT_SALES = "Latitude"
LON_SALES = "Longitude"

# === Load DataCo Sales ===
print("Loading DataCo daily sales...")
sales_df = pd.read_csv(SALES_FILE)

# Parse and normalize date
sales_df[SALES_DATE_COL] = pd.to_datetime(sales_df[SALES_DATE_COL], errors='coerce').dt.normalize()

# Rename lat/lon to match weather files
sales_df = sales_df.rename(columns={LAT_SALES: "latitude", LON_SALES: "longitude"})

# Round lat/lon for matching
sales_df["latitude"] = sales_df["latitude"].round(3)
sales_df["longitude"] = sales_df["longitude"].round(3)

print("\n=== Sample DataCo Coordinates ===")
print(sales_df[['latitude','longitude']].drop_duplicates().head())

# === Process each year ===
for year in YEARS:
    weather_file = os.path.join(WEATHER_DIR, f"visualcrossing_weather_{year}_cleaned.csv")
    if not os.path.exists(weather_file):
        print(f"\nSkipping {year} - weather file not found")
        continue

    print(f"\n=== Processing Year {year} ===")
    weather_df = pd.read_csv(weather_file)

    # Parse and normalize weather date
    weather_df[WEATHER_DATE_COL] = pd.to_datetime(weather_df[WEATHER_DATE_COL], errors='coerce').dt.normalize()

    # Round weather lat/lon for matching
    weather_df["latitude"] = weather_df["latitude"].round(3)
    weather_df["longitude"] = weather_df["longitude"].round(3)

    print(f"Weather sample coords for {year}:", weather_df[['latitude','longitude']].drop_duplicates().head().values.tolist())

    # Filter sales data for current year
    sales_year = sales_df[sales_df[SALES_DATE_COL].dt.year == year].copy()

    # Merge on date + lat/lon
    merged = pd.merge(
        sales_year,
        weather_df,
        left_on=[SALES_DATE_COL, "latitude", "longitude"],
        right_on=[WEATHER_DATE_COL, "latitude", "longitude"],
        how="left"
    ).drop(columns=[WEATHER_DATE_COL])

    # Save merged file
    output_file = os.path.join(OUTPUT_DIR, f"DataCo_Weather_{year}_Merged.csv")
    merged.to_csv(output_file, index=False)
    print(f"Saved merged dataset for {year}: {output_file} (Rows: {merged.shape[0]})")


Loading DataCo daily sales...

=== Sample DataCo Coordinates ===
   latitude  longitude
0    39.565    -87.289
1    18.226    -66.047
2    18.245    -66.371
3    40.286    -75.089
4    18.228    -66.044

=== Processing Year 2015 ===
Weather sample coords for 2015: [[17.982, -66.113], [18.007, -66.636], [18.018, -66.616], [18.025, -66.613], [18.025, -66.615]]
Saved merged dataset for 2015: ./merged_data_final\DataCo_Weather_2015_Merged.csv (Rows: 3090)

=== Processing Year 2016 ===
Weather sample coords for 2016: [[17.982, -66.113], [18.007, -66.636], [18.025, -66.613], [18.025, -66.615], [18.033, -66.852]]
Saved merged dataset for 2016: ./merged_data_final\DataCo_Weather_2016_Merged.csv (Rows: 1755)

=== Processing Year 2017 ===
Weather sample coords for 2017: [[-33.938, 18.571], [17.982, -66.113], [18.007, -66.636], [18.018, -66.616], [18.025, -66.613]]
Saved merged dataset for 2017: ./merged_data_final\DataCo_Weather_2017_Merged.csv (Rows: 7994)


In [3]:
# Concatenates DataCo + Weather merged files (2015–2017) into one dataset

import pandas as pd
import os

# === Config ===
MERGED_DIR = "./merged_data_final"    # Folder containing year-wise merged files
OUTPUT_FILE = "./final_datasets/DataCo_Weather_2015_2017_All.csv"
os.makedirs("./final_datasets", exist_ok=True)

# File paths
file_2015 = os.path.join(MERGED_DIR, "DataCo_Weather_2015_Merged.csv")
file_2016 = os.path.join(MERGED_DIR, "DataCo_Weather_2016_Merged.csv")
file_2017 = os.path.join(MERGED_DIR, "DataCo_Weather_2017_Merged.csv")

# Load and concatenate all years
print("Loading all year-wise merged files...")
df_2015 = pd.read_csv(file_2015)
df_2016 = pd.read_csv(file_2016)
df_2017 = pd.read_csv(file_2017)

all_df = pd.concat([df_2015, df_2016, df_2017], ignore_index=True)

# Save the combined dataset
all_df.to_csv(OUTPUT_FILE, index=False)
print(f"Saved combined dataset: {OUTPUT_FILE} (Rows: {all_df.shape[0]})")


Loading all year-wise merged files...
Saved combined dataset: ./final_datasets/DataCo_Weather_2015_2017_All.csv (Rows: 12839)


In [4]:
# Adds lag features (1, 7, 30 days) for Sales and Temperature to DataCo + Weather dataset.

import pandas as pd
import os

# === Config ===
INPUT_FILE = "./final_datasets/DataCo_Weather_2015_2017_All.csv"
OUTPUT_FILE = "./final_datasets/DataCo_Weather_Lagged.csv"

# Columns to lag
TARGET_COLS = ["Sales", "temp"]  # Change "Sales" if your sales column has a different name

# Lag periods (in days)
LAGS = [1, 7, 30]

# Load dataset
print("Loading combined dataset...")
df = pd.read_csv(INPUT_FILE)

# Ensure date is datetime
df["date_only"] = pd.to_datetime(df["date_only"], errors='coerce')

# Sort by location and date
df = df.sort_values(by=["latitude", "longitude", "date_only"])

# Generate lag features for each target variable
for col in TARGET_COLS:
    for lag in LAGS:
        df[f"{col}_lag_{lag}"] = df.groupby(["latitude", "longitude"])[col].shift(lag)

# Save output
df.to_csv(OUTPUT_FILE, index=False)
print(f"Saved dataset with Sales + Temp lag features: {OUTPUT_FILE} (Rows: {df.shape[0]})")


Loading combined dataset...
Saved dataset with Sales + Temp lag features: ./final_datasets/DataCo_Weather_Lagged.csv (Rows: 12839)
